<a href="https://colab.research.google.com/github/rprimi/IR_bm25/blob/main/proj0_is_bm25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Projeto de construção de um sistema de recuperação de informação para seleção na disciplina da Unicamp "Deep Learning aplicado a sistemas de buscas"

> Indented block

